# note-0-just-test
This notebook is for just testing the inference of a model.
It loads the full-trained model for demonstration and test it on the MNIST dataset.

## Flow
1. Install the dependencies
1. Preset the environment
1. Load the model
1. Test the model

## Step

### 1. Install the dependencies

In [ ]:
!pip install --quiet --upgrade pip
!pip install --quiet -r requirements.txt

# Preset

In [ ]:
import os, requests
import matplotlib.pyplot as plt
import torch, torchvision

# Config
## env
FORCE_CPU = True
SAMPLE_MODEL_URL = "https://github.com/jyje/pilot-mlops-cicd/raw/c4c9955a4cdcc62fedf26c9365200e4a6fbec048/assets/sample_model.pt"

## logic
CHANNEL_SIZE = 3
RESIZE_SIZE = (224, 224)

NORMALIZE_MEAN = (0.5,0.5,0.5)
NORMALIZE_STD = (0.5,0.5,0.5)

DATA_ROOT_PATH = "./data"
MODEL_ROOT_PATH = "./models"
MODEL_NAME = "pilot"
MODEL_VERSION = "1"

# Hyperparameters
learning_rate = 0.001
momentum = 0.9
num_epochs = 3
batch_size = 100
num_workers = 0 # main process
t_max = 200

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(RESIZE_SIZE),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Lambda(lambda x: x.repeat(CHANNEL_SIZE, 1, 1)),
        torchvision.transforms.Normalize(NORMALIZE_MEAN,NORMALIZE_STD),
    ]
)

if FORCE_CPU:
    device = torch.device("cpu")
    print("CPU is forced. Using CPU.")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. torch.cuda is the API for NVIDIA GPU support")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("MPS is available and built. torch.mps is the API for Apple Silicon GPU.")
else:
    device = torch.device("cpu")
    print("Neither CUDA nor MPS is available or built. Using CPU.")

# Get Dataset

In [ ]:
dataset_test = torchvision.datasets.MNIST(root=DATA_ROOT_PATH, train=False, download=True, transform=transform)
dataloader_test = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=True, num_workers=num_workers)

## Load Model

In [ ]:
if os.path.exists(f"{MODEL_ROOT_PATH}/{MODEL_NAME}/{MODEL_VERSION}/model.pt"):
    print("Model already exists. Loading the model...")
else:
    print("Model not found. Downloading the model...")
    os.makedirs(f"{MODEL_ROOT_PATH}/{MODEL_NAME}/{MODEL_VERSION}", exist_ok=True)

    response = requests.get(SAMPLE_MODEL_URL)
    with open(f"{MODEL_ROOT_PATH}/model.pt", "wb") as f:
        f.write(response.content)

loaded_model = torch.jit.load(f"{MODEL_ROOT_PATH}/{MODEL_NAME}/{MODEL_VERSION}/model.pt")

loaded_model.eval()
loaded_model = loaded_model.to(device)

## Test Model

In [ ]:
# Visualize a sample
sample_test = next(iter(dataloader_test))

img_test = sample_test[0][0].permute(1, 2, 0) # (C, H, W) -> (H, W, C)
img_test = (img_test + 1) / 2 # Normalize Grayscale to [0, 1]

test_img = img_test.permute(2, 0, 1).unsqueeze(dim=0).to(device)  # (H,W,C) -> (1,C,H,W)
response = loaded_model(test_img)
probabilities = torch.nn.functional.softmax(response, dim=1)
predicted_class = torch.argmax(probabilities).item()

plt.imshow(img_test)
plt.title(f"Testing sample (Real: {sample_test[1][0]}, Predicted: {predicted_class})")
plt.show()

print(f"Predicted Number: {predicted_class}")
print(f"Probability Distribution:")
for idx, prob in enumerate(probabilities[0]):
    print(f" - Number {idx}: {prob.item()*100:.2f}%")